In [10]:
import pandas as pd
import re
import guess_language

In [2]:
# load data
scrapeDf = pd.read_pickle('pkl/scraped_podcasts_COMPLETE.pkl')
apiDf = pd.read_pickle('pkl/itunes_podcasts_v3_COMPLETE.pkl')

In [3]:
# clean column names because SQL doesn't like camelCase
def convert(name):
    """Converts camelCase to camel_case."""
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()

scrapeDf.columns = [convert(x) for x in scrapeDf.columns]
apiDf.columns = [convert(x) for x in apiDf.columns]

In [11]:
podcastDf = pd.merge(apiDf, scrapeDf, how = 'inner', on = 'collection_id')

In [15]:
podcastDf.shape

(6284, 40)

In [14]:
# convert to int
podcastDf['collection_id'] = [int(x) for x in podcastDf['collection_id']]

In [16]:
# guess language of summary
podcastDf['language'] = [guess_language.guessLanguageName(x) for x in podcastDf['podcast_summary']]
podcastDf = podcastDf[podcastDf['language'] == 'English']
podcastDf.shape

(5950, 41)

In [36]:
podcastDf.columns

Index([               u'artist_id',              u'artist_name',
                u'artist_view_url',           u'artwork_url100',
                  u'artwork_url30',            u'artwork_url60',
                 u'artwork_url600', u'collection_censored_name',
        u'collection_explicitness',      u'collection_hd_price',
                  u'collection_id',          u'collection_name',
               u'collection_price',      u'collection_view_url',
        u'content_advisory_rating',                  u'country',
                       u'currency',                 u'feed_url',
                      u'genre_ids',                   u'genres',
                             u'id',                     u'kind',
             u'primary_genre_name',        u'radio_station_url',
                   u'release_date',      u'track_censored_name',
                    u'track_count',       u'track_explicitness',
                 u'track_hd_price',    u'track_hd_rental_price',
                       u'

In [37]:
episodeDf = podcastDf[['collection_id','episode_descriptions', 'episode_names']]

In [39]:
# convert to list
episodeDf['episode_descriptions'] = [x.split("\',") for x in episodeDf['episode_descriptions']]
episodeDf['episode_names'] = [x.split("\',") for x in episodeDf['episode_names']]

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [59]:
test_description = episodeDf.iloc[0:10]['episode_descriptions']

In [68]:
def clean_text(text):
    # remove mixed alphanumeric
    text = re.sub(r"""(?x) # verbose regex
                            \b    # Start of word
                            (?=   # Look ahead to ensure that this word contains...
                             \w*  # (after any number of alphanumeric characters)
                             \d   # ...at least one digit.
                            )     # End of lookahead
                            \w+   # Match the alphanumeric word
                            \s*   # Match any following whitespace""", 
                             "", text)
    
    # remove urls
    text = re.sub(r'\s([\S]*.com[\S]*)\b', '', text)
    text = re.sub(r'\s([\S]*.org[\S]*)\b', '', text)

    
    
    # remove non-alphanumeric, non-space
    text = re.sub(r'([^\s\w]|_)+', '', text)
    
    
    
    return text

In [75]:
clean_episode_description = []
for pod in episodeDf['episode_descriptions']:
    clean_list = []
    for ep in pod:
        clean_list.append(clean_text(ep))
    clean_episode_description.append(clean_list)

#episodeDf['episode_descriptions'] = [clean_text(ep) for pod in episodeDf['episode_descriptions'] for ep in pod]
    

In [78]:
episodeDf['clean_episode_description'] = clean_episode_description

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [80]:
clean_episode_name = []
for pod in episodeDf['episode_names']:
    clean_list = []
    for ep in pod:
        clean_list.append(clean_text(ep))
    clean_episode_name.append(clean_list)

In [81]:
episodeDf['clean_episode_name'] = clean_episode_name

/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [82]:
episodeDf.head()

,collection_id,episode_descriptions,episode_names,clean_episode_description,clean_episode_name
0,515836681,"[['In this episode of ""A Cast of Kings,"" Joann...","[['A Cast of Kings S1E10 - Fire and Blood, 'A...",[In this episode of A Cast of Kings Joanna and...,"[A Cast of Kings Fire and Blood, A Cast of K..."
1,126723118,"[['Insurance news, interviews, rating announce...","[['BestDay - Friday, January 22, 2016, 'BestD...",[Insurance news interviews rating announcement...,"[BestDay Friday January , BestDay Thursday..."
2,699748055,"[[""They said they'd do it and they're followin...","[['Embrace the Spoilers: ""Star Wars: The Force...",[They said theyd do it and theyre following th...,[Embrace the Spoilers Star Wars The Force Awak...
3,80693391,[['Adultery is a very real danger to every mar...,"[['""How to Divorce-Proof Your Marriage - II"", ...",[Adultery is a very real danger to every marri...,"[How to DivorceProof Your Marriage II, How t..."
4,579605641,"[['hai, toiukotode, Jin Hui de10Hui Mu noHeng ...","[['yuku2015Nian kuru2016Nian ~ He Zheng , 'ar...",[hai toiukotode Jin Hui Mu noHeng Li Ji Li Yua...,"[ He Zheng , Nian Te Fan woShu Hui niDu tsute..."


In [83]:
del episodeDf['episode_descriptions']
del episodeDf['episode_names']

In [84]:
podcastDf.columns

Index([               u'artist_id',              u'artist_name',
                u'artist_view_url',           u'artwork_url100',
                  u'artwork_url30',            u'artwork_url60',
                 u'artwork_url600', u'collection_censored_name',
        u'collection_explicitness',      u'collection_hd_price',
                  u'collection_id',          u'collection_name',
               u'collection_price',      u'collection_view_url',
        u'content_advisory_rating',                  u'country',
                       u'currency',                 u'feed_url',
                      u'genre_ids',                   u'genres',
                             u'id',                     u'kind',
             u'primary_genre_name',        u'radio_station_url',
                   u'release_date',      u'track_censored_name',
                    u'track_count',       u'track_explicitness',
                 u'track_hd_price',    u'track_hd_rental_price',
                       u'

In [85]:
del podcastDf['episode_descriptions']
del podcastDf['episode_names']

In [86]:
podcastDf = pd.merge(podcastDf, episodeDf, how = 'inner', on='collection_id')

In [89]:
# clean podcast summary
podcastDf['podcast_summary'] = [clean_text(x) for x in podcastDf['podcast_summary']]

In [92]:
# save clean data
podcastDf.to_pickle('pkl/clean_podcast_data.pkl')